# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [16]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('C:/Users/Sergei/Desktop/Python Materials/Python_7_pandas_functions_groupby/ml-latest-small/movies.csv')
rating = pd.read_csv('C:/Users/Sergei/Desktop/Python Materials/Python_7_pandas_functions_groupby/ml-latest-small/ratings.csv')

In [3]:
rating = rating.groupby('movieId').mean()

In [4]:
movies = movies.merge(rating, on='movieId') 

In [5]:
movies.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,338.558704,3.872470,1.103116e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,318.906542,3.401869,1.069321e+09
2,3,Grumpier Old Men (1995),Comedy|Romance,374.423729,3.161017,9.662429e+08
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,355.538462,2.384615,9.277797e+08
4,5,Father of the Bride Part II (1995),Comedy,320.785714,3.267857,9.967201e+08


In [6]:
def classify(row):
    if row['rating'] <= 2:
        return 'Low'
    elif row['rating'] <= 4:
        return 'Mid'
    else:
        return 'High'

movies['class'] = movies.apply(classify, axis=1)
movies[['title','class']].head()

,title,class
0,Toy Story (1995),Mid
1,Jumanji (1995),Mid
2,Grumpier Old Men (1995),Mid
3,Waiting to Exhale (1995),Mid
4,Father of the Bride Part II (1995),Mid


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [7]:
keywords = pd.read_csv('C:/Users/Sergei/Desktop/Python Materials/Python_7_pandas_functions_groupby/keywords.csv')

In [8]:
keywords[keywords.keyword == 'москва']

,keyword,shows
1259,москва,153050


In [9]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [35]:
def geoclass(row):
    q = row['keyword']
    
    for (key, value) in geo_data.items():
        if q in value:
            return key
        else:
            continue
    return 'undefined'

keywords['region'] = keywords.apply(geoclass, axis=1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [36]:
keywords[keywords['region'] != 'undefined']

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток
11222,псков,24885,Северо-Запад
11785,тула,23898,Центр
20823,петербург,14859,Северо-Запад
21445,сахалин,14494,Дальний Восток


In [12]:
for (key, value) in geo_data.items():
    print('2. key', key)
    print('2. value', value)

2. key Центр
2. value ['москва', 'тула', 'ярославль']
2. key Северо-Запад
2. value ['петербург', 'псков', 'мурманск']
2. key Дальний Восток
2. value ['владивосток', 'сахалин', 'хабаровск']


# Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [21]:
years = np.arange(1950,2010+1).tolist()

In [37]:
def production_year(row):
    for yyyy in years:
        if str(yyyy) in row['title']:
            return yyyy
        else:
            continue
    return 1900

movies['year'] = movies.apply(production_year, axis=1)
movies.head()

,movieId,title,genres,userId,rating,timestamp,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,338.558704,3.872470,1.103116e+09,Mid,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,318.906542,3.401869,1.069321e+09,Mid,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,374.423729,3.161017,9.662429e+08,Mid,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,355.538462,2.384615,9.277797e+08,Mid,1995
4,5,Father of the Bride Part II (1995),Comedy,320.785714,3.267857,9.967201e+08,Mid,1995


In [39]:
results = movies[['year','rating']]

In [43]:
results.groupby('year').mean().sort_values(by='rating',ascending=False)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295
